In [2]:
import mlflow
import pandas as pd
from mlflow import MlflowClient

In [3]:
REMOTE_TRACKING_URI = "http://ec2-44-217-145-52.compute-1.amazonaws.com:5000"
experiment_name = "base_pattern"
mlflow.set_tracking_uri(REMOTE_TRACKING_URI)
client = MlflowClient(REMOTE_TRACKING_URI)

experiment = mlflow.get_experiment_by_name(experiment_name)
mlflow.set_experiment(experiment_id=experiment.experiment_id)

<Experiment: artifact_location='s3://kaggle-nakayama/mlflow/6', creation_time=1730853199485, experiment_id='6', last_update_time=1730853199485, lifecycle_stage='active', name='base_pattern', tags={}>

In [ ]:
run_id = "0add35cfa1224f2995bffa12ae61e17b"
parent_run = mlflow.get_run(run_id=run_id)

In [20]:
runs = mlflow.search_runs(
    filter_string=f"tags.mlflow.parentRunId='{run_id}'",
    order_by=["tags.mlflow.runName DESC"],
).loc[
    :,
    [
        "start_time",
        "end_time",
        "metrics.qini_at_60",
        "metrics.qini_at_80",
        "metrics.uplift_at_40",
        "metrics.auuc",
        "metrics.qini_at_50",
        "metrics.uplift_at_0",
        "metrics.uplift_at_30",
        "metrics.uplift_at_90",
        "metrics.qini_at_90",
        "metrics.uplift_at_20",
        "metrics.uplift_at_60",
        "metrics.uplift_at_80",
        "metrics.uplift_at_50",
        "metrics.qini_at_70",
        "metrics.uplift_at_70",
        "metrics.uplift_at_10",
        "metrics.qini_at_0",
        "metrics.qini_at_30",
        "metrics.qini_at_40",
        "metrics.qini_at_20",
        "metrics.qini_at_10",
        "tags.mlflow.runName",
        "tags.model",
    ],
]

In [21]:
runs

,start_time,end_time,metrics.qini_at_60,metrics.qini_at_80,metrics.uplift_at_40,metrics.auuc,metrics.qini_at_50,metrics.uplift_at_0,metrics.uplift_at_30,metrics.uplift_at_90,...,metrics.qini_at_70,metrics.uplift_at_70,metrics.uplift_at_10,metrics.qini_at_0,metrics.qini_at_30,metrics.qini_at_40,metrics.qini_at_20,metrics.qini_at_10,tags.mlflow.runName,tags.model
0,2024-11-10 11:30:39.946000+00:00,2024-11-10 11:34:27.567000+00:00,614.280687,790.914148,0.011730,3.966157,567.220009,0.0,0.012765,0.008940,...,697.122155,0.009650,0.019195,0.0,397.275050,485.266627,290.123318,200.108597,lenta_xlearner,xlearner
1,2024-11-10 11:48:13.406000+00:00,2024-11-10 11:50:51.012000+00:00,645.728922,829.137140,0.011906,3.920319,545.357308,0.0,0.013206,0.010293,...,714.807895,0.009887,0.015606,0.0,410.896398,493.872571,323.722827,162.071535,lenta_tlearner,tlearner
2,2024-11-10 11:45:46.058000+00:00,2024-11-10 11:48:12.511000+00:00,723.661233,971.604081,0.014308,4.515509,702.977352,0.0,0.014039,0.010867,...,863.193344,0.011910,0.018849,0.0,437.371014,592.857629,222.971685,196.332130,lenta_slearner,slearner
3,2024-11-10 11:34:28.415000+00:00,2024-11-10 11:45:45.165000+00:00,531.385134,773.723209,0.010079,2.404516,455.098559,0.0,0.011006,0.009351,...,712.499101,0.009861,0.014004,0.0,342.122139,417.117708,204.456413,145.133886,lenta_rlearner,rlearner
4,2024-11-10 11:26:26.265000+00:00,2024-11-10 11:30:39.046000+00:00,708.739436,812.150550,0.011443,2.129321,562.920146,0.0,0.009340,0.010011,...,741.440441,0.010265,0.005616,0.0,290.635131,473.845495,137.658374,58.355031,lenta_drlearner,drlearner
